## Clound ML Engine으로 2층 CNN 학습하기

### 사전 준비
- ML Engine API 활성화 하기
https://console.cloud.google.com/apis/api/ml.googleapis.com/overview
- cloudml-magic 확장 프로그램 설치하기
- 다음 코드 셀 **!pip install cloudmlmagic**을 실행하고, 노트 세션을 리셋합니다.
- **일단은 datalab에서 구동 확인**

In [2]:
! pip install cloudmlmagic

  Using cached https://files.pythonhosted.org/packages/e5/7d/9d5a640c4f8bf2c8b1afc015e9a9d8de32e13c9016dcc4b0ec03481fb396/uritemplate-3.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/e5/38518af393f7c214357079ce67a317307936896e961e35450b70fad2a9cf/rsa-4.0-py2.py3-none-any.whl
  Running setup.py bdist_wheel for cloudmlmagic: started
  Running setup.py bdist_wheel for cloudmlmagic: finished with status 'done'
  Stored in directory: C:\Users\박찬우\AppData\Local\pip\Cache\wheels\b2\55\88\fbb2119a451eb8fedac05ac630eeda698f34e29fbfa2b21d86
  Running setup.py bdist_wheel for httplib2: started
  Running setup.py bdist_wheel for httplib2: finished with status 'done'
  Stored in directory: C:\Users\박찬우\AppData\Local\pip\Cache\wheels\6d\41\4b\2b369d6e2b7eaebcdd423516d3fb659c7658c16a2be8fd04ec
Successfully built cloudmlmagic httplib2


노트북 세션을 리셋 해 주세요  
다음 셀을 리셋 후에 실행해 주세요

In [3]:
#확장 프로그램 읽어 들이기
%load_ext cloudmlmagic

The cloudmlmagic module is not an IPython extension.


- 프로젝트 ID, GCS 버킷을 설명합니다.

- <font color="red">**PROJECTID**, **BUCKET**</font>을 변경해주세요.

In [4]:
%ml_init -projectId PROJECTID -bucket BUCKET -region asia-east1 -scaleTier BASIC_GPU

UsageError: Line magic function `%ml_init` not found.


다음 코드는 ML Engine으로 실행됩니다.  
일반적으로 몇 분 만에 처리가 끝납니다. 하지만 리소스 확보를 못하면 가끔 오류가 발생하기도 합니다.  
이 때는 다시 실행해주세요.

In [ ]:
%%ml_run cloud
# 이 셀 코드는 ML Engine으로 실행됩니다.

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(\"./mnist/\")

batch_size = 50

with tf.Graph().as_default():
    X = tf.placeholder(tf.float32, [None, 784], name='X')
    y = tf.placeholder(tf.float32, [None, ], name='y')

    X_image = tf.reshape(X, [-1, 28, 28, 1])

    # 합성곱층 1번째
    conv1 = tf.layers.conv2d(
        inputs=X_image,
        filters=32,
        kernel_size=[5, 5],
        padding=\"SAME\",
        activation=tf.nn.relu)

    # 풀링층 1번째
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
 
    # 합성곱층 1번째
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding=\"SAME\",
        activation=tf.nn.relu)
 
    # 풀링층 2번째
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
 
    # 전결합층
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(
        inputs=pool2_flat, units=1024, activation=tf.nn.relu)
 
    # 드롭아웃층
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.5, training=True)
 
    # 출력층
    logits = tf.layers.dense(inputs=dropout, units=10, name='output')
    predict = tf.argmax(logits, 1)
 
    # 손실
    with tf.name_scope('calc_loss'):
        onehot_labels = tf.one_hot(indices=tf.cast(y, tf.int32), depth=10)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=onehot_labels, logits=logits, name='xentropy')
        loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
 
    # 손실 최소화 하기
    train_op = tf.train.AdamOptimizer(0.0001).minimize(loss)
 
    # 정답률 계산하기
    with tf.name_scope('calc_accuracy'):
        correct_prediction = tf.equal(
            tf.argmax(logits, 1), tf.argmax(onehot_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        total_batch = int(mnist.train.num_examples // batch_size)
        for epoch in range(20):
            for step in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                _, loss_value = sess.run([train_op, loss],
                                         feed_dict={X: batch_xs, y: batch_ys})
            print('Step: %d, Loss: %f' % (step, loss_value))
    
        # test
        _a = sess.run(accuracy, feed_dict={
                      X: mnist.test.images, y: mnist.test.labels})
            print('Accuracy: %f' % _a)

결과는 다음 URL에서 확인할 수 있습니다.  
https://console.cloud.google.com/mlengine/jobs